# Segmenting and Clustering Neighbourhoods in Toronto

## Question 1

Scrape Wikipedia page to obtain a list of postal codes in Canada where the first letter is M. Postal codes beginning with M are located within the city of Toronto in the province of Ontario. Only the first three characters are listed, corresponding to the Forward Sortation Area.

Wikipedia page: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M,<br />
BeautifulSoup Web scraping package: http://beautiful-soup-4.readthedocs.io/en/latest/

In [1]:
!pip install BeautifulSoup4
!pip install tabulate

### Scrape Wikipedia page to obtain a list of postal codes

In [2]:
from bs4 import BeautifulSoup
import requests

url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url,'xml')

table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')

data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

data[:10]


[[],
 ['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Harbourfront'],
 ['M6A', 'North York', 'Lawrence Heights'],
 ['M6A', 'North York', 'Lawrence Manor'],
 ['M7A', 'Downtown Toronto', "Queen's Park"],
 ['M8A', 'Not assigned', 'Not assigned']]

### Create and Clean dataframe:

1. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood<br />
2. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.<br />
3. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.<br />
4. More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will
   notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined
   into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [3]:
import pandas 

# 1. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
df = pandas.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]  

# 2. Ignore cells with a borough that is Not assigned.
df = df.drop(df[(df.Borough == "Not assigned")].index)

# 3. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
df.Neighbourhood.replace("Not assigned", df.Borough, inplace=True)

# 4. Duplicate neighborhood over Two rows will be combined into one row with the neighborhoods separated with a comma
df = df.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

df.head(10)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


5. In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.


In [4]:
df.shape

(103, 3)

## Question 2

Use the Geocoder package or the csv file to create the following dataframe
Link to csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [7]:
df2=pandas.read_csv('http://cocl.us/Geospatial_data')
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merge both data frames using PostalCode as key

In [8]:

# Rename Column name to match df dataframe
df2.rename(columns={'Postal Code':'PostalCode'}, inplace=True)

# Merge both df2 and df dataframe on PostalCode to new dataframe df3
df3 = pandas.merge(df2, df, on='PostalCode')

# Insert Header
df4 = df3[['PostalCode','Borough','Neighbourhood','Latitude','Longitude']]

df4.head(20)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [9]:
df4.shape

(103, 5)

# Question 3

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:
to generate maps to visualize your neighborhoods and how they cluster together.

### Map of PostCodes

In [10]:
# plotting library
import folium 

toronto_map = folium.Map(location=[43.651070,-79.347015],zoom_start=11)

for lat,lng,borough,neighbourhood in zip(df4['Latitude'],df4['Longitude'],df4['Borough'],df4['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(toronto_map)

toronto_map

### FourSquare API - Get RestaurantType by PostCode

In [11]:

# Foursquare API Credentials
CLIENT_ID = 'PQ02G3AXW2KUQRNFY0LY2B5UCEBYMXZUQTIN34KYUQEMWM4H'
CLIENT_SECRET = '4AEPF0U2MYDA0JIBBCYEA3ASYQQZQ4SNRPMXYTFLR22W3DWT'
API_VERSION = '20200101'

# Foursquare API Settings
LIMIT = 200  
RADIUS = 300 
CAT_ID = '4d4b7105d754a06374d81259'  # Category: Food


# RestaurantType Dataframe
df_restaurant_type = pandas.DataFrame(columns=['RestaurantType','PostalCode'])

# Iterate PostalCode df
for (index, row) in df4.iterrows():
    
    # build API request URL
    url = \
        'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
        CLIENT_ID,
        CLIENT_SECRET,
        API_VERSION,
        row['Latitude'],
        row['Longitude'],
        RADIUS,
        LIMIT,
        CAT_ID,
        )
        
    # GET request into result DataFrame
    df_result = pandas.DataFrame(list(venue['categories'][0]['name'] for venue in requests.get(url).json()['response']['venues']), columns=['RestaurantType'])

    # Empty then populate with 'No Restaurants'
    if df_result.shape[0] == 0:
        df_result.loc[0, ['RestaurantType']] = 'No Restaurants'
        
    df_result['PostalCode'] = row['PostalCode']
    df_restaurant_type = pandas.concat([df_restaurant_type, df_result])
    
# Reset Index    
df_restaurant_type.reset_index(drop=True, inplace=True)


In [12]:
# Check FourSquare data
print('Shape:', df_restaurant_type.shape)
df_restaurant_type.head(10)

Shape: (1565, 2)


,RestaurantType,PostalCode
0,No Restaurants,M1B
1,No Restaurants,M1C
2,Restaurant,M1E
3,Bakery,M1E
4,Chinese Restaurant,M1E
5,Restaurant,M1E
6,Korean Restaurant,M1G
7,Thai Restaurant,M1H
8,Hakka Restaurant,M1H
9,Caribbean Restaurant,M1H


In [13]:
# Merge into df4 on PostalCode
df5 = pandas.merge(df4, df_restaurant_type, on='PostalCode')
df5.head(10)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,RestaurantType
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,No Restaurants
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,No Restaurants
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Restaurant
3,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Bakery
4,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Chinese Restaurant
5,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Restaurant
6,M1G,Scarborough,Woburn,43.770992,-79.216917,Korean Restaurant
7,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,Thai Restaurant
8,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,Hakka Restaurant
9,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,Caribbean Restaurant


### Clustering Toronto neighbourhoods by restaurant types

In [14]:
# One-hot encoding
df_cluster = df5.copy()
df_cluster = df_cluster.drop('RestaurantType', axis=1).join(pandas.get_dummies(df_cluster[['RestaurantType']], prefix='', prefix_sep=''))

print(df_cluster.shape)
df_cluster.head(5)

(1565, 127)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Afghan Restaurant,African Restaurant,American Restaurant,Asian Restaurant,BBQ Joint,...,Tapas Restaurant,Tea Room,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# Group by postal codes
df_cluster = df_cluster.groupby(['PostalCode', 'Neighbourhood', 'Latitude', 'Longitude']).sum().reset_index()

print(df_cluster.shape)
df_cluster.head()

(103, 126)


,PostalCode,Neighbourhood,Latitude,Longitude,Afghan Restaurant,African Restaurant,American Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,...,Tapas Restaurant,Tea Room,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint
0,M1B,"Rouge, Malvern",43.806686,-79.194353,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M1C,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M1E,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M1G,Woburn,43.770992,-79.216917,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M1H,Cedarbrae,43.773136,-79.239476,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [16]:
from tabulate import tabulate

# Number of top restaurant types
top_count = 10  
top_restaurant_types = df_cluster[list(df_cluster.iloc[:, 5:])].sum(axis=0).reset_index().sort_values(by=0, ascending=False).iloc[:top_count]['index'].values.tolist()

print(tabulate(pandas.DataFrame({'Top 10 restaurant types': top_restaurant_types}).set_index('Top 10 restaurant types'), headers='keys', tablefmt='psql'))

+---------------------------+
| Top 10 restaurant types   |
|---------------------------|
| Coffee Shop               |
| Café                      |
| Restaurant                |
| Pizza Place               |
| Italian Restaurant        |
| Bakery                    |
| Sandwich Place            |
| Fast Food Restaurant      |
| Japanese Restaurant       |
| Breakfast Spot            |
+---------------------------+


### K-Means Clustering

Divide Toronto neighbourhoods into groups based on top 10 types of restaurants located in the neighbourhood

In [17]:
from sklearn.cluster import KMeans

df_kmeans = df_cluster[top_restaurant_types]

k = 3
kmeans = KMeans(n_clusters=k, random_state=3).fit(df_kmeans)
df_cluster.insert(2, 'Cluster', kmeans.labels_)

df_cluster['Cluster']=kmeans.labels_
df_cluster.head(5)

,PostalCode,Neighbourhood,Cluster,Latitude,Longitude,Afghan Restaurant,African Restaurant,American Restaurant,Asian Restaurant,BBQ Joint,...,Tapas Restaurant,Tea Room,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint
0,M1B,"Rouge, Malvern",0,43.806686,-79.194353,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M1C,"Highland Creek, Rouge Hill, Port Union",0,43.784535,-79.160497,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M1E,"Guildwood, Morningside, West Hill",0,43.763573,-79.188711,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M1G,Woburn,0,43.770992,-79.216917,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M1H,Cedarbrae,0,43.773136,-79.239476,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [18]:
# print clusters in tabular format
print(tabulate(df_cluster['Cluster'].value_counts().sort_index().to_frame(),headers=['Cluster number','Neighbourhoods in the cluster'], tablefmt='psql', colalign=('center', 'center')))

+------------------+---------------------------------+
|  Cluster number  |  Neighbourhoods in the cluster  |
|------------------+---------------------------------|
|        0         |               80                |
|        1         |               17                |
|        2         |                6                |
+------------------+---------------------------------+


### Visualize clusters in map

In [19]:

# plotting library
import folium 

cluster_map = folium.Map(location=[43.651070,-79.347015],zoom_start=11)

# set color scheme for the clusters
colours = ['green', 'blue', 'red']

# add markers to the map
markers_colors = []

for (lat, lng, pc, nbh, cluster) in zip(df_cluster['Latitude'], df_cluster['Longitude'], 
                                        df_cluster['PostalCode'], df_cluster['Neighbourhood'], df_cluster['Cluster']):
    label = folium.Popup(str(pc) + ': ' + str(nbh) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=colours[cluster],
        fill=True,
        fill_color=colours[cluster],
        fill_opacity=0.6,
        ).add_to(cluster_map)

cluster_map


### Map Key

In [20]:
for colour in sorted(colours, reverse=True):
    df = df_cluster.loc[df_cluster['Cluster'] == colours.index(colour)].copy()
    df.drop([
        'PostalCode',
        'Neighbourhood',
        'Latitude',
        'Longitude',
        'Cluster',
        ], axis=1, inplace=True)
    Rcount = df.sum().sum()
    Ncount = df.shape[0]
    N0count = df['No Restaurants'].sum()
    RperN = int(round(Rcount / (Ncount - N0count)))
    N0percentage = int(round(N0count * 100 / Ncount))
    print('''Cluster: {}
Restaurants in the cluster: {}
Neighbourhoods in the cluster: {}
Percentage of neighbourhoods without restaurants: {}%
Restaurants per neighbourhood: {}'''.format(colour.title(),
            Rcount, Ncount, N0percentage, RperN))
    print(tabulate(df.sum().sort_values(ascending=False).head().to_frame(),
           headers=['Top restaurant types in the cluster', 'Count'
           ], tablefmt='psql'), '\n')

Cluster: Red
Restaurants in the cluster: 299
Neighbourhoods in the cluster: 6
Percentage of neighbourhoods without restaurants: 0%
Restaurants per neighbourhood: 50
+---------------------------------------+---------+
| Top restaurant types in the cluster   |   Count |
|---------------------------------------+---------|
| Coffee Shop                           |      95 |
| Restaurant                            |      21 |
| Café                                  |      17 |
| Food Court                            |      13 |
| Deli / Bodega                         |      12 |
+---------------------------------------+---------+ 

Cluster: Green
Restaurants in the cluster: 471
Neighbourhoods in the cluster: 80
Percentage of neighbourhoods without restaurants: 26%
Restaurants per neighbourhood: 8
+---------------------------------------+---------+
| Top restaurant types in the cluster   |   Count |
|---------------------------------------+---------|
| Coffee Shop                           |